In [2]:
import pandas as pd
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
from collections import defaultdict
import math

In [3]:
college_statistics = pd.read_csv('finalData.csv')
data = college_statistics

In [4]:
data.dtypes

Unnamed: 0             int64
name                  object
team                  object
rnd                    int64
pick                   int64
pos                   object
year                   int64
heightinchestotal    float64
weight                 int64
fortyyd              float64
twentyss             float64
threecone            float64
vertical             float64
broad                  int64
bench                  int64
school                object
conference            object
games                  int64
rushingAtt           float64
rushingYds           float64
rushingAvg           float64
rushingTD            float64
passCmp              float64
passAtt              float64
passPct              float64
passYds              float64
passTD               float64
passInt              float64
passRate             float64
recYds               float64
recAtt               float64
recTD                float64
recAvg               float64
soloTackles          float64
tackleAssists 

In [10]:
data.conference.unique()

array(['ACC', 'Pac-12', nan, 'SEC', 'Big 12', 'Big Ten', 'American',
       'Big East', 'Pac-10', 'MAC', 'Sun Belt', 'MWC', 'WAC', 'CUSA',
       'Ind', 'Southern', 'MVC', 'Pac-8', 'Big West', 'SWC', 'Southland',
       'Big 8', 'SWAC'], dtype=object)

In [19]:
rows = []
for index, row in data.iterrows():
    x = row.values
    x = np.append(x[1:], [row['heightinchestotal'] * row['weight'], row['rushingAtt'] * row['rushingYds'], row['passTD'] / (row['passInt'] + 1), row['passRate'] * row['passPct'], row['rushingAvg'] * row['rushingTD'], row['rushingAtt'] * row['rushingYds'], row['recAvg'] * row['recTD'], row['recAtt'] * row['recYds'], row['soloTackles'] * row['tackleAssists'], row['totalTackles'] * row['sacks'], row['intTDs'] * row['ints'], row['passDef'] * row['ints'], row['fumblesForced'] * row['totalTackles']])
    rows.append(x)
    if index % 50 == 0:
        print index

comboFeats = ['name', 'team', 'rnd', 'pick', 'pos', 'year', 'heightinchestotal', 'weight', 'fortyyd', 'twentyss', 'threecone', 'vertical', 'broad', 'bench', 'school', 'conference', 'games', 'rushingAtt', 'rushingYds', 'rushingAvg', 'rushingTD', 'passCmp', 'passAtt', 'passPct', 'passYds', 'passTD', 'passInt', 'passRate', 'recYds', 'recAtt', 'recTD', 'recAvg', 'soloTackles', 'tackleAssists', 'totalTackles', 'sacks', 'ints', 'intTDs', 'passDef', 'fumbles', 'fumblesForced', 'heightinchestotal * weight', 'rushingAtt * rushingYds', 'passTD / passInt', 'passRate * passPct', 'rushingAvg * rushingTD', 'rushingAtt * rushingYds', 'recAvg * recTD', 'recAtt * recYds', 'soloTackles * tackleAssists', 'totalTackles * sacks', 'intTDs * ints', 'passDefs * ints', 'fumblesForced * totalTackles']

dataframe = pd.DataFrame(data=rows, columns=comboFeats)
dataframe.to_csv('combinedFeatures.csv')
# 'heightinchestotal * weight', 'rushingAtt * rushingYds', 'passTD / passInt', 'passRate * passPct', 'rushingAvg * rushingTD', 'rushingAtt * rushingYds', 'recAvg * recTD', 'recAtt * recYds', 'soloTackles * tackleAssists', 'totalTackles * sacks', 'intTDs * ints', 'passDefs * ints', 'fumblesForced * totalTackles',
# heightinchestotal * weight
# rushingAtt * rushingYds
# passTD / passInt
# passRate * passPct
# rushingAvg * rushingTD
# rushingAtt * rushingYds
# recAvg * recTD
# recAtt * recYds
# soloTackles * tackleAssists
# totalTackles * sacks
# intTDs * ints
# passDefs * ints
# fumblesForced * totalTackles

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800
3850
3900
3950
4000
4050
4100
4150
4200
4250
4300


In [11]:
# need to figure out how to add 'school', 'conference' back in. 
features = ['heightinchestotal', 'weight', 'fortyyd', 'vertical', 'bench', 'twentyss', 'threecone', 'broad', 
            'games', 'rushingAtt', 'rushingYds', 'rushingAvg', 'rushingTD', 'passCmp',
            'passPct', 'passYds', 'passTD', 'passInt', 'passRate', 'recYds', 'recAtt', 'recTD', 'recAvg', 'soloTackles', 
            'tackleAssists', 'totalTackles', 'sacks', 'ints', 'intTDs', 'passDef', 'fumbles', 'fumblesForced']

In [12]:
discreteFeatures = []
for val in data.conference.unique():
    discreteFeatures.append('confUnknown' if pd.isnull(val) else 'conf' + val)

featureCutoffs = {}
for feature in features:
    featureCutoffs[feature] = {}
    sli = data[(pd.notnull(data[feature]) & data[feature] != 0)]
    desc = sli.describe()
    featureCutoffs[feature]['25%'] = desc[feature]['25%']
    featureCutoffs[feature]['50%'] = desc[feature]['50%']
    featureCutoffs[feature]['75%'] = desc[feature]['75%']
    discreteFeatures.append(feature+'None')
    discreteFeatures.append(feature+'Q1') 
    discreteFeatures.append(feature+'Q2') 
    discreteFeatures.append(feature+'Q3') 
    discreteFeatures.append(feature+'Q4')
    
print discreteFeatures

['confACC', 'confPac-12', 'confUnknown', 'confSEC', 'confBig 12', 'confBig Ten', 'confAmerican', 'confBig East', 'confPac-10', 'confMAC', 'confSun Belt', 'confMWC', 'confWAC', 'confCUSA', 'confInd', 'confSouthern', 'confMVC', 'confPac-8', 'confBig West', 'confSWC', 'confSouthland', 'confBig 8', 'confSWAC', 'heightinchestotalNone', 'heightinchestotalQ1', 'heightinchestotalQ2', 'heightinchestotalQ3', 'heightinchestotalQ4', 'weightNone', 'weightQ1', 'weightQ2', 'weightQ3', 'weightQ4', 'fortyydNone', 'fortyydQ1', 'fortyydQ2', 'fortyydQ3', 'fortyydQ4', 'verticalNone', 'verticalQ1', 'verticalQ2', 'verticalQ3', 'verticalQ4', 'benchNone', 'benchQ1', 'benchQ2', 'benchQ3', 'benchQ4', 'twentyssNone', 'twentyssQ1', 'twentyssQ2', 'twentyssQ3', 'twentyssQ4', 'threeconeNone', 'threeconeQ1', 'threeconeQ2', 'threeconeQ3', 'threeconeQ4', 'broadNone', 'broadQ1', 'broadQ2', 'broadQ3', 'broadQ4', 'gamesNone', 'gamesQ1', 'gamesQ2', 'gamesQ3', 'gamesQ4', 'rushingAttNone', 'rushingAttQ1', 'rushingAttQ2', 'rus

In [ ]:
rows = []
i = 0
for index, row in data.iterrows():
    i += 1
    if i % 50 == 0:
        print i
    playerDiscreteFeatures = set()
    playerDiscreteFeatures.add('confUnknown' if pd.isnull(row['conference']) else 'conf' + row['conference'])
    for feature in features:
        if pd.isnull(row[feature]) or row[feature] == 0:
            playerDiscreteFeatures.add(feature + 'None')
        elif row[feature] < featureCutoffs[feature]['25%']:
            playerDiscreteFeatures.add(feature + 'Q1')
        elif row[feature] < featureCutoffs[feature]['50%']:
            playerDiscreteFeatures.add(feature + 'Q2')
        elif row[feature] < featureCutoffs[feature]['75%']:
            playerDiscreteFeatures.add(feature + 'Q3')
        elif row[feature] >= featureCutoffs[feature]['75%']:
            playerDiscreteFeatures.add(feature + 'Q4')

    featureArray = [row['name'], row['rnd'], row['pick'], row['pos'], row['year']]
    for discreteFeature in discreteFeatures:
        featureArray.append(1 if discreteFeature in playerDiscreteFeatures else 0)
    rows.append(featureArray)
    
    discreteDF = pd.DataFrame(data=rows, columns=['name', 'rnd', 'pick', 'pos', 'year'] + discreteFeatures)
    discreteDF.to_csv('test.csv')

In [82]:
test = discreteDF.head(256)
test.to_csv('discreteTest.csv')
training = discreteDF.tail(discreteDF.shape[0] - 256)
training.to_csv('discreteTrain.csv')

In [4]:
shouldTransformFeatures = False 
MAX_YEAR = 2015 
MIN_YEAR = 1999

In [5]:
# get player values/draft positions based on 'nfl_draft' data
def get_player_draft_info():
    valuesMap = {}
    pickMap = {}
    MAX_VALUE = 256 # number of draft picks + 1 (7 rounds * 32 picks + 32 compensatory + 1)
    for index, row in data.iterrows():
        primaryKey = (row['name'], row['year'])
        valuesMap[primaryKey] = (MAX_VALUE - row['pick'])
        pickMap[primaryKey] = row['pick']
    return valuesMap, pickMap

In [6]:
# Creates list of positions to partition by
positionsSet = set()
for position in data['pos']:
    positionsSet.add(position)
positions = list(positionsSet)

In [7]:
draftValues, draftPositions = get_player_draft_info()

In [8]:
# replace here with NEW dataset 
combineFeatures = data[['name', 'pos', 'year'] + features]

In [9]:
def transformPercentile(dataFrame, colName):
    newCol = dataFrame[colName].rank(pct=True) * 100 - 50
    dataFrame.update(pd.Series(newCol, name=colName))

In [10]:
def get_position_maps(shouldTransform):
    positionMaps = {}
    for position in positions:
        positionMaps[position] = combineFeatures[combineFeatures['pos'] == position]
        if shouldTransform:
            for feature in features:
                transformPercentile(positionMaps[position], feature)
    return positionMaps

In [11]:
positionMaps = get_position_maps(shouldTransformFeatures)

In [12]:
positionMaps

{'C':                   name pos  year  heightinchestotal  weight  fortyyd  \
 18       CameronErving   C  2015               77.0     313    -5.15   
 60           AliMarpet   C  2015               76.0     307    -4.98   
 70        HronissGrasu   C  2015               75.0     297    -4.52   
 130          ShaqMason   C  2015               73.0     207    -4.52   
 132          MaxGarcia   C  2015               76.0     309    -4.52   
 207         AndyGallik   C  2015               74.0     306    -5.50   
 221       AustinReiter   C  2015               73.0     207    -4.52   
 298     WestonRichburg   C  2014               75.0     298    -5.10   
 325       MarcusMartin   C  2014               75.0     320    -4.52   
 331      TravisSwanson   C  2014               77.0     312    -5.28   
 348      BrandonLinder   C  2014               78.0     311    -5.35   
 360         BryanStork   C  2014               76.0     315    -4.52   
 366      RussellBodine   C  2014             

In [13]:
# build X matrix and Y values numpy arrays 
def build_data_arrays(test_year):
    trainX = defaultdict(list)
    testX = defaultdict(list)
    trainY = defaultdict(list)
    names = defaultdict(list)
    testCount = 0
    trainCount = 0
    for position in positionMaps:
        for index, row in positionMaps[position].iterrows(): 
            if not row['year'] == test_year:
                trainCount += 1
                trainX[position].append(row[2:])                  
                trainY[position].append(draftValues[(row['name'], row['year'])])
            if row['year'] == test_year:
                testCount += 1
                testX[position].append(row[2:])
                names[position].append(row['name'])
    print 'Total test rows =', testCount
    print 'Total train rows =', trainCount
    return trainX, testX, trainY, names

In [14]:
lr = linear_model.LinearRegression()

In [15]:
# Map players to their true and predicted draft positions 
def get_average_error(results, test_year):
    trueDraftMap = {}      # true pick number
    predictedDraftMap = {} # relative pick based on regression
    i = 0
    for index, row in results.iterrows():
        trueDraftMap[row['name']] = draftPositions[(row['name'], test_year)] #need primary key to get draft positions 
        predictedDraftMap[row['name']] = i
        i += 1
    
    # Compute absolute_error based on relative draft positions
    i = 0
    absolute_error = 0
    errors = {}
    for key, value in sorted(trueDraftMap.iteritems(), key=lambda (k,v): (v,k)):  
        errors[key] = math.fabs(i - predictedDraftMap[key]) 
        absolute_error += errors[key]
        i += 1
        
    #Add the errors to the results Dataframe (for visualization)
    results['error'] = 0.0
    for index, row in results.iterrows():
        results.at[index, 'error'] = errors[row['name']]

    #average the error by the total number of players
    return absolute_error / float(len(results))

In [16]:
def run_linear_regression(positions, trainX, trainY, testX, names, test_year):
    errorMap = {} 
    output = {} 
    coefficients = {}
    lr = linear_model.LinearRegression()
    for position in positions:
        X = np.array(trainX[position])
        Y = np.array(trainY[position])
        testArray = np.array(testX[position])
        if len(testArray) > 0 and len(X) > 0 and len(Y) > 0:
            lr.fit(X, Y)
            coefficients[position] = pd.DataFrame(zip(features, lr.coef_), columns = ['feature', 'coefficient']).sort_values(by=['coefficient'], ascending=False)
            prediction = lr.predict(testArray)
            output[position] = pd.DataFrame(zip(names[position], prediction), columns = ['name', 'value']).sort_values(by=['value'], ascending=False)
            errorMap[position] = get_average_error(output[position], test_year)
    return errorMap, output, coefficients

In [17]:
def get_train_error(test_year, validation_year):
    current_year = MIN_YEAR
    aggregateMap = {}
    while current_year <= MAX_YEAR:
        trainX = defaultdict(list)
        testX = defaultdict(list)
        trainY = defaultdict(list)
        names = defaultdict(list)
        testCount = 0
        trainCount = 0
        for position in positionMaps:
            for index, row in positionMaps[position].iterrows():
                if not row['year'] == test_year or not row['year'] == validation_year:
                    trainCount += 1
                    trainX[position].append(row[2:])                  
                    trainY[position].append(draftValues[(row['name'], row['year'])])
                    if row['year'] == current_year:
                        testCount += 1
                        testX[position].append(row[2:])
                        names[position].append(row['name'])
        print 'Total test rows in {} = {}'.format(current_year, testCount)
        print 'Total train rows = {}'.format(trainCount)
        errors, results, coefficients = run_linear_regression(positions, trainX, trainY, testX, names, current_year)
        if current_year == MIN_YEAR:
            aggregateMap = errors
        else:
            for key, value in errors.iteritems():
                if key in aggregateMap:
                    aggregateMap[key] += value
        current_year += 1
    averageMap = {}
    for (key, value) in aggregateMap.iteritems():
        averageMap[key] = value / float(MAX_YEAR - MIN_YEAR)
    return averageMap

In [18]:
def get_test_error_and_results(test_year):
    trainX, testX, trainY, names = build_data_arrays(test_year)
    errors, results, coefficients = run_linear_regression(positions, trainX, trainY, testX, names, test_year)
    return errors, results, coefficients

In [19]:
averageMap = get_train_error(2015, -1) # -1 means no validation year 

Total test rows in 1999 = 253
Total train rows = 4332
Total test rows in 2000 = 254
Total train rows = 4332
Total test rows in 2001 = 246
Total train rows = 4332
Total test rows in 2002 = 261
Total train rows = 4332
Total test rows in 2003 = 262
Total train rows = 4332
Total test rows in 2004 = 255
Total train rows = 4332
Total test rows in 2005 = 255
Total train rows = 4332
Total test rows in 2006 = 255
Total train rows = 4332
Total test rows in 2007 = 255
Total train rows = 4332
Total test rows in 2008 = 252
Total train rows = 4332
Total test rows in 2009 = 256
Total train rows = 4332
Total test rows in 2010 = 255
Total train rows = 4332
Total test rows in 2011 = 254
Total train rows = 4332
Total test rows in 2012 = 253
Total train rows = 4332
Total test rows in 2013 = 254
Total train rows = 4332
Total test rows in 2014 = 256
Total train rows = 4332
Total test rows in 2015 = 256
Total train rows = 4332


In [20]:
averageMap

{'C': 1.3175595238095237,
 'DB': 12.597885268512723,
 'DE': 5.336906909889143,
 'DT': 5.19206837808454,
 'FB': 0.4010416666666667,
 'G': 3.8755079529071232,
 'K': 0.0625,
 'LB': 7.881617375820597,
 'P': 0.0,
 'QB': 2.7965334910187853,
 'RB': 4.321363354861527,
 'T': 5.331458051322603,
 'TE': 3.0317220989346274,
 'WR': 7.739510464498605}

In [21]:
TEST_YEAR = 2015
errors, results, coefficients = get_test_error_and_results(TEST_YEAR)

Total test rows = 256
Total train rows = 4076


In [22]:
errors

{'C': 1.4285714285714286,
 'DB': 8.304347826086957,
 'DE': 5.523809523809524,
 'DT': 7.363636363636363,
 'FB': 1.5,
 'G': 6.0,
 'LB': 8.166666666666666,
 'P': 0.0,
 'QB': 2.0,
 'RB': 2.4444444444444446,
 'T': 5.3,
 'TE': 4.8,
 'WR': 8.647058823529411}

In [23]:
results['RB']

,name,value,error
1,MelvinGordon,205.621653,1.0
0,ToddGurley,169.429284,1.0
4,TevinColeman,162.549037,2.0
3,AmeerAbdullah,159.637776,0.0
5,DukeJohnson,156.345491,1.0
9,JavoriusAllen,155.203225,4.0
12,JayAjayi,142.077413,6.0
8,JeremyLangford,116.508124,1.0
7,MattJones,105.782766,1.0
15,JoshRobinson,104.126031,6.0


In [24]:
coefficients['RB']

,feature,coefficient
3,vertical,2.220393e+02
7,broad,9.579643e+01
31,fumblesForced,6.127726e+01
22,recAvg,2.969603e+01
25,totalTackles,2.922335e+01
13,passCmp,1.786621e+01
26,sacks,1.617611e+01
11,rushingAvg,2.641714e+00
20,recAtt,2.196953e+00
2,fortyyd,1.624950e+00
